In [1]:
import sys

sys.path.append("..")

In [2]:
from util.run import solve

In [9]:
domain = """
agent(alice;bob).
time(0..10).
amount(0..100).
commodity("EUR";"good").
action(payer_payee_amount_commodity(Payer, Payee, Amount, Commodity)) :-
  agent(Payer), agent(Payee),
  amount(Amount),
  commodity(Commodity).
"""

obs = """
obs(agent_account_balance_commodity(alice, register, 100, "EUR"), 0).
obs(agent_account_balance_commodity(alice, store, 0, "goods"), 0).
obs(agent_account_balance_commodity(bob, register, 0, "EUR"), 0).
obs(agent_account_balance_commodity(bob, store, 1, "goods"), 0).


obs(agent_account_balance_commodity(alice, commitments, 0, Obligation), 0) :-
  Payment = payer_payee_amount_commodity(alice, bob, 100, "EUR"),
  Obligation = debtor_creditor_action_start_end(alice, bob, Payment, 4, 10).
  
obs(agent_account_balance_commodity(bob, demands, 0, Obligation), 0) :-
  Payment = payer_payee_amount_commodity(alice, bob, 100, "EUR"),
  Obligation = debtor_creditor_action_start_end(alice, bob, Payment, 4, 10).

obs(agent_account_balance_commodity(bob, commitments, 0, Obligation), 0) :-
  Payment = payer_payee_amount_commodity(bob, alice, 1, "goods"),
  Obligation = debtor_creditor_action_start_end(bob, alice, Payment, 4, 10).
  
obs(agent_account_balance_commodity(alice, demands, 0, Obligation), 0) :-
  Payment = payer_payee_amount_commodity(bob, alice, 1, "goods"),
  Obligation = debtor_creditor_action_start_end(bob, alice, Payment, 4, 10).

"""

occs = """

occ(contract_agreement, 1).
occ(payer_payee_amount_commodity(alice, bob, 100, "EUR"), 3).
occ(payer_payee_amount_commodity(bob, alice, 1, "good"), 6).

"""

contract_rules = """

occ(Obligation, T) :-
  Payment = payer_payee_amount_commodity(alice, bob, 100, "EUR"),
  Obligation = debtor_creditor_action_start_end(alice, bob, Payment, T+1, T+7),
  occ(contract_agreement, T).
  
occ(Obligation, T) :-
  Payment = payer_payee_amount_commodity(bob, alice, 1, "good"),
  Obligation = debtor_creditor_action_start_end(bob, alice, Payment, T+1, T+7),
  occ(contract_agreement, T).

"""

obligation_rules = """

occ(credit(Debtor, commitments, 1, Obligation), T) :- 
  Obligation = debtor_creditor_action_start_end(Debtor, Creditor, Action, Start, End),
  occ(Obligation, T).
  
occ(debit(Creditor, demands, 1, Obligation), T) :- 
  Obligation = debtor_creditor_action_start_end(Debtor, Creditor, Action, Start, End),
  occ(Obligation, T).
  
"""

obligation_fulfilment_rules = """

occ(debit(Debtor, commitments, 1, Obligation), T) :-
    occ(Action, T),
    Start <= T, T <= End,
    Obligation = debtor_creditor_action_start_end(Debtor, Creditor, Action, Start, End).

occ(credit(Creditor, demands, 1, Obligation), T) :-
    occ(Action, T),
    Start <= T, T <= End,
    Obligation = debtor_creditor_action_start_end(Debtor, Creditor, Action, Start, End).

"""

payment_rules = """

occ(debit(Payee, register, Amount, Commodity), T) :-
  occ(Payment, T),
  Payment = payer_payee_amount_commodity(Payer, Payee, Amount, Commodity),
  RAmount >= Amount,
  obs(agent_account_balance_commodity(Payer, register, RAmount, Commodity), T).
  
occ(debit(Payee, store, Amount, Commodity), T) :-
  occ(Payment, T),
  Payment = payer_payee_amount_commodity(Payer, Payee, Amount, Commodity),
  RAmount >= Amount,
  obs(agent_account_balance_commodity(Payer, store, RAmount, Commodity), T).

occ(credit(Payer, register, Amount, Commodity), T) :-
  occ(Payment, T),
  Payment = payer_payee_amount_commodity(Payer, Payee, Amount, Commodity),
  RAmount >= Amount,
  obs(agent_account_balance_commodity(Payer, register, RAmount, Commodity), T).

occ(credit(Payer, store, Amount, Commodity), T) :-
  occ(Payment, T),
  Payment = payer_payee_amount_commodity(Payer, Payee, Amount, Commodity),
  RAmount >= Amount,
  obs(agent_account_balance_commodity(Payer, store, RAmount, Commodity), T).

"""

accounts = """

account_type(demands;register;store, active).
account_type(commitments, passive).

"""

balance_inertia_rules = """

obs(agent_account_balance_commodity(Agent, Account, Balance, Commodity), T+1) :-
  obs(agent_account_balance_commodity(Agent, Account, Balance, Commodity), T),
  time(T),
  not occ(debit(Agent, Account, _, Commodity), T), not occ(credit(Agent, Account, _, Commodity), T).

"""

transaction_rules = """

%%% AKTIVE

% DEBIT
obs(agent_account_balance_commodity(Agent, Account, Balance + Amount, Commodity), T+1) :-
  obs(agent_account_balance_commodity(Agent, Account, Balance, Commodity), T),
  occ(debit(Agent, Account, Amount, Commodity), T),
  time(T),
  account_type(Account, active).

% CREDIT
obs(agent_account_balance_commodity(Agent, Account, Balance - Amount, Commodity), T+1) :-
  obs(agent_account_balance_commodity(Agent, Account, Balance, Commodity), T),
  occ(credit(Agent, Account, Amount, Commodity), T),
  time(T),
  account_type(Account, active).

%%% PASSIVE

% DEBIT
obs(agent_account_balance_commodity(Agent, Account, Balance - Amount, Commodity), T+1) :-
  obs(agent_account_balance_commodity(Agent, Account, Balance, Commodity), T),
  occ(debit(Agent, Account, Amount, Commodity), T),
  time(T),
  account_type(Account, passive).

% CREDIT
obs(agent_account_balance_commodity(Agent, Account, Balance + Amount, Commodity), T+1) :- 
  obs(agent_account_balance_commodity(Agent, Account, Balance, Commodity), T),
  occ(credit(Agent, Account, Amount, Commodity), T),
  time(T),
  account_type(Account, passive).

"""

show = """
%#show obs(agent_account_balance_commodity(alice, account, 0;1;100, Commodity), T) : time(T), commodity(Commodity).
#show obs/2.
#show occ/2.
"""

In [10]:
solve([domain,
       obs,
       occs,
       #contract_rules,
       #obligation_rules,
       #obligation_fulfilment_rules,
       payment_rules,
       accounts,
       balance_inertia_rules,
       transaction_rules,
       show,
      ], sep='\n');

Answer  1: {
obs(agent_account_balance_commodity(alice,commitments,0,debtor_creditor_action_start_end(alice,bob,payer_payee_amount_commodity(alice,bob,100,"EUR"),4,10)),0)
obs(agent_account_balance_commodity(alice,commitments,0,debtor_creditor_action_start_end(alice,bob,payer_payee_amount_commodity(alice,bob,100,"EUR"),4,10)),1)
obs(agent_account_balance_commodity(alice,commitments,0,debtor_creditor_action_start_end(alice,bob,payer_payee_amount_commodity(alice,bob,100,"EUR"),4,10)),2)
obs(agent_account_balance_commodity(alice,commitments,0,debtor_creditor_action_start_end(alice,bob,payer_payee_amount_commodity(alice,bob,100,"EUR"),4,10)),3)
obs(agent_account_balance_commodity(alice,commitments,0,debtor_creditor_action_start_end(alice,bob,payer_payee_amount_commodity(alice,bob,100,"EUR"),4,10)),4)
obs(agent_account_balance_commodity(alice,commitments,0,debtor_creditor_action_start_end(alice,bob,payer_payee_amount_commodity(alice,bob,100,"EUR"),4,10)),5)
obs(agent_account_balance_commodi